In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import datetime

t0 = datetime.now()
#url = 'https://test2.buildwhiz.com/bw-3.01/etc/BIDataConnector?key=sdg'
url = 'https://www.buildwhiz.com/bw-3.01/etc/BIDataConnector?key=sdg'
response = requests.get(url)
print('HTTP Status:', response.status_code)
dfs = {}
if response.status_code == 200:
  content = response.content
  print('HTTP content Length:', len(content))
  #bs = BeautifulSoup(content, 'html.parser')
  bs = BeautifulSoup(content, 'lxml')
  headings = [h2.text.lower() for h2 in bs.find_all('h2')]
  print('HTTP headings:', headings)
  html_tables = bs.find_all('table')
  print('No of html-tables found:', len(html_tables))
  converters = {'s': lambda str: str, 'csv': lambda str: [p.strip() for p in str.split(',')], 
      'i': int, 'b': lambda str: str == 'true', 'f': float, 'l': np.int64}
  for heading, types, tbl in [(table['id'], table['types'], table) for table in  html_tables]:
    tr_array = tbl.find_all('tr')
    col_names = [td.text for td in tr_array[0].find_all('td')]
    cfs = [converters[typ] for typ in types.split(',')]
    data_rows = [[cf(td.text) if td.text else None for cf, td in zip(cfs, tr.find_all('td'))] for tr in tr_array[1:]]
    print("%s[%s] (%d): %r" % (heading, types, len(data_rows), col_names))
    dfs[heading] = pd.DataFrame(data_rows, columns=col_names)
  delay = datetime.now() - t0
  print('timedelta:', delay)
else:
  print('Quitting due to HTTP error')